In [1]:
import numpy as np
import pandas as p

#documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_test.csv")
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_entities.csv")
promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/promoted_content.csv", dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/sample_submission.csv")
#documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_topics.csv")
train = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : str, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views_sample.csv")

In [3]:
#add number of ads on display
train_ad_count_per_display = train.groupby(['display_id'])['display_id'].agg({'ads_on_doc' : 'count'}).reset_index()
train = train.merge(train_ad_count_per_display, how = 'left', on = 'display_id')
del train_ad_count_per_display

In [8]:
#add the document_id describing ad
train = train.merge(promoted_content.drop(promoted_content.columns[[2,3]],axis = 1), how = 'left', on = 'ad_id')
#add the document_id of the display
train = train.merge(events.drop(events.columns[[1,3,4,5]],axis = 1), how = 'left', on = 'display_id', suffixes = ('_of_ad',''))

In [27]:
#create document_on_ad, calculating the statistical strenght of the ad document link
document_on_ad = train.groupby(['document_id','ad_id','document_id_of_ad','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
document_on_ad['clicked_percent'] = document_on_ad['clicked'] / document_on_ad['total']
#all pairs of document_on_ad that show at least 10 times together
document_on_ad_10 = document_on_ad[document_on_ad.total > 10]
#add expected chance of ad to be chosen [uni_chance]
document_on_ad_10['uni_chance'] = document_on_ad_10.apply(lambda row: 1 / row.ads_on_doc, axis=1)

In [75]:
#just to see that 40% of the quantity of ad_doc lines repeat over 10 times 
#(that's without grouping the different ad_on_doc of same pairs)
float(document_on_ad_10.total.sum()) / document_on_ad.total.sum()

0.3935121623875018

In [77]:
#create a likelihood column which shows how much stronger or weaker 
#the actual clicked percentage is in relation to the uniform chance
document_on_ad_10['likelihood'] = document_on_ad_10.apply(lambda row: row['clicked_percent'] / row['uni_chance'], axis=1)

/Users/astrachan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [83]:
#on the way to adding together same pairs with different ad_on_doc
document_on_ad_10['like_mul_total'] = document_on_ad_10.apply(lambda row: row['likelihood'] * row['total'], axis=1)
scores = document_on_ad_10.drop(document_on_ad_10.columns[[2,3,5,6,7,8]],axis = 1).groupby(['document_id','ad_id']).sum().reset_index()

/Users/astrachan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [93]:
#table giving a score to ad document pairing, can be used later on
#also to make sets of different entities, categories and topics
scores['score'] = scores.apply(lambda row: row['like_mul_total'] / row['total'], axis=1)

In [98]:
scores.sort_values('score',inplace=True)